In [1]:
import os
import zipfile
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [65]:
if not os.path.exists('train_df.csv'):
    with zipfile.ZipFile('../2_feature_engineering/train_df.zip', 'r') as zip_ref:
        zip_ref.extractall('.')    

train_df = pd.read_csv('train_df.csv')
train_df = train_df.drop(['Unnamed: 0'], axis = 1)
train_df.head(3)

,id,001,01,05,10,100,11,12,12 healthy,13,...,Steele VR,Eckhardt AL,Biessmann F,Oh A,Horvath JC,Carter O,Sanz-Leon P,Rice GE,Braun U,semantic_cluster
0,15850739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,20
1,28077311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,37
2,23968735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,25


In [66]:
if not os.path.exists('test_df.csv'):
    with zipfile.ZipFile('../2_feature_engineering/test_df.zip', 'r') as zip_ref:
        zip_ref.extractall('.')    


test_df = pd.read_csv('test_df.csv')
test_df = test_df.drop(['Unnamed: 0'], axis = 1)

test_df.head(3)

,id,001,01,05,10,100,11,12,12 healthy,13,...,Steele VR,Eckhardt AL,Biessmann F,Oh A,Horvath JC,Carter O,Sanz-Leon P,Rice GE,Braun U,semantic_cluster
0,23964193,0.0,0.062086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,42
1,16650777,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,20663646,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,41


In [67]:
X_train = train_df.drop(columns='citation_count')
y_train = train_df['citation_count']

X_test = test_df.drop(columns='citation_count')
y_test = test_df['citation_count']

In [68]:
cat_features = [col for col in X_train.columns if 'journal_' in col and col not in ['journal_impact_factor', 'journal_h_index']]

with open('../2_feature_engineering/prominent_prevalent_authors.pkl', 'rb') as f:
    authors = list(pickle.load(f))

author_features = [col for col in X_train.columns if col in authors]

cat_features.extend(author_features)

Scaling

In [69]:
scaler = StandardScaler()

numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_features.remove('id')

numeric_features = [item for item in numeric_features if item not in cat_features]

X_train_numeric = X_train[numeric_features]
X_test_numeric = X_test[numeric_features]

X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

X_train_numeric_scaled = pd.DataFrame(X_train_numeric_scaled, columns=X_train_numeric.columns)
X_test_numeric_scaled = pd.DataFrame(X_test_numeric_scaled, columns=X_test_numeric.columns)

# Replace the numeric columns in X_train and X_test with the scaled values
X_train[X_train_numeric.columns] = X_train_numeric_scaled
X_test[X_test_numeric.columns] = X_test_numeric_scaled

<b><u>Feature Selection:</u></b>

In [70]:
abstracts_df = pd.read_csv('../1_eda/abstracts/abstracts_df_eda.csv')
abstracts_df = abstracts_df.drop(['Unnamed: 0'], axis = 1)

abstracts_df.head(3)

,id,001,01,05,10,100,11,12,12 healthy,13,...,young,young adults,young healthy,young older,younger,younger adults,youth,zone,citation_count,journal
0,9065511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,the journal of neuroscience : the official jou...
1,9084599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464.0,journal of neurophysiology
2,9114263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,367.0,journal of neurophysiology


In [71]:
abstracts_correlated_feature_selection_columns = [col for col in abstracts_df.columns if col not in ['id', 'journal', 'title', 'authors', 'citation_count']]

In [10]:
def get_correaltion_df(df: pd.DataFrame) -> pd.DataFrame:
    correlation_matrix = df.corr()

    correlation_df = correlation_matrix[correlation_matrix > 0.7].unstack().reset_index()
    correlation_df.columns = ['Feature1', 'Feature2', 'Correlation']

    # Drop the rows where Feature1 and Feature2 are the same
    correlation_df = correlation_df[correlation_df['Feature1'] != correlation_df['Feature2']]
    correlation_df = correlation_df.dropna(subset=['Correlation'])

    correlation_df['AbsCorrelation'] = correlation_df['Correlation'].abs()
    correlation_df = correlation_df.sort_values('AbsCorrelation', ascending=False)

    correlation_df['FeaturesTuple'] = correlation_df.apply(lambda row: tuple(sorted([row['Feature1'], row['Feature2']])), axis=1)

    # Drop duplicates based on the sorted tuple
    correlation_df = correlation_df.drop_duplicates(subset=['FeaturesTuple'])
    correlation_df = correlation_df.drop(columns=['FeaturesTuple'])

    return correlation_df


In [11]:
def get_correlated_columns_to_drop(correlation_df: pd.DataFrame) -> list[str]:
    columns_to_drop = []
    for index, row in correlation_df.iterrows():
        feature1 = row['Feature1']
        feature2 = row['Feature2']

        # Split the feature names into words
        feature1_words = feature1.split(' ')
        feature2_words = feature2.split(' ')

        # If both features are equally expressive, drop feature2
        if len(feature1_words) == len(feature2_words):
            columns_to_drop.append(feature2)
        else:
            # Otherwise, drop the less expressive feature
            if len(feature1_words) > len(feature2_words):
                columns_to_drop.append(feature2)
            else:
                columns_to_drop.append(feature1)

    return columns_to_drop        

In [12]:
correlation_df = get_correaltion_df(abstracts_df[abstracts_correlated_feature_selection_columns])

correlation_df

,Feature1,Feature2,Correlation,AbsCorrelation
6911144,periodicals,wiley periodicals,1.000000,1.000000
2240751,cross sectional,sectional,1.000000,1.000000
4062341,globus pallidus,pallidus,1.000000,1.000000
7007651,planum temporale,temporale,1.000000,1.000000
962287,basal ganglia,ganglia,0.999305,0.999305
...,...,...,...,...
2707762,disorder ocd,obsessive,0.704427,0.704427
7213337,precentral,precentral gyrus,0.704337,0.704337
835388,attention deficit,adhd,0.704096,0.704096
4429297,hyperactivity disorder,hyperactivity,0.703673,0.703673


In [22]:
correlated_columns_to_drop = get_correlated_columns_to_drop(correlation_df)

correlated_columns_to_drop

['periodicals',
 'sectional',
 'pallidus',
 'temporale',
 'ganglia',
 'opercularis',
 'positron',
 'mapp',
 'working',
 'globus',
 'pallidus',
 'deficit hyperactivity',
 'conductance',
 'oxygen level',
 'wiley',
 'wiley',
 'corpus',
 'emission tomography',
 'callosum',
 'resonance',
 'tegmental',
 'positron',
 'obsessive',
 'callosum',
 'accumbens',
 'attention deficit',
 'planum',
 'temporale',
 'oxygenation',
 'oxygenation level',
 'emission',
 'hyperactivity disorder',
 'wiley',
 'parkinson',
 'grey',
 'hum',
 'emission',
 'magnetic',
 'courses',
 'heschl',
 'mapp',
 'gray',
 'periodicals',
 'hum mapp',
 'emission',
 'mapp',
 'alzheimer',
 'mapp',
 'mapp',
 'white',
 'thickness',
 'resting',
 'transcranial magnetic',
 'magnetic',
 'heart',
 'supplementary',
 'occipito',
 'basal',
 'obsessive compulsive',
 'basal',
 'hoc',
 'anisotropy',
 'arterial',
 'oxygenation',
 'morphometry',
 'making',
 'wiley',
 'tensor',
 'angular',
 'referential',
 'periodicals',
 'hum',
 'functional magnet

In [18]:
abstracts_feature_selection_columns = [col for col in abstracts_df.columns if col not in ['id', 'journal', 'title', 'authors', 'citation_count'] and col not in correlated_columns_to_drop]
X_train_abstracts_feature_selection = X_train[abstracts_feature_selection_columns]

estimator = RandomForestRegressor(n_estimators=2, random_state=0, verbose=2)

selector = RFE(estimator, n_features_to_select=1000, step=2, verbose=3)
selector = selector.fit(X_train_abstracts_feature_selection, y_train)

prominent_features = [f for f,s in zip(X_train_abstracts_feature_selection.columns, selector.support_) if s]

import csv

with open('prominent_features.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(prominent_features)

Fitting estimator with 3032 features.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 2


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


building tree 2 of 2


...

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   47.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 2


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.9s remaining:    0.0s


building tree 2 of 2


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   47.6s finished


In [72]:
prominent_features = pd.read_csv('prominent_features.csv', header=None, dtype=str).T[0].tolist()

In [73]:
abstract_df_dropped_columns = [col for col in abstracts_df.columns if col not in prominent_features and col not in ['id', 'journal', 'title', 'authors', 'citation_count'] and col not in correlated_columns_to_drop]
len(abstract_df_dropped_columns)

2032

In [74]:
columns_to_drop = correlated_columns_to_drop + abstract_df_dropped_columns

In [82]:
len(set(columns_to_drop))

2225

In [77]:
X_train = X_train.drop(columns_to_drop, axis = 1)
X_test = X_test.drop(columns_to_drop, axis = 1)

In [78]:
X_train = X_train.drop(['id'], axis = 1)
X_test = X_test.drop(['id'], axis = 1)

In [84]:
X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')


with zipfile.ZipFile('X_train.zip', 'w') as zf:
    zf.write('X_train.csv')

with zipfile.ZipFile('X_test.zip', 'w') as zf:
    zf.write('X_test.csv')


os.remove('X_train.csv')    
os.remove('X_test.csv')   